# Python Sets: Introduction & Internal Mechanics

A **Set** in Python is a built-in data structure that represents an **unordered collection of unique elements**.

Mathematically, it mirrors a finite set found in mathematics. From an engineering perspective, it is implemented as a **Hash Table** (similar to a Dictionary, but with keys only and no values). This implementation detail dictates its primary characteristics: extremely fast membership testing () but a complete lack of order.

---

## 1. Key Characteristics

To use sets effectively, you must understand these four pillars:

1. **Unordered:** Sets do **not** record element position or insertion order.
* *Implication:* You cannot access items via an index (e.g., `s[0]` raises `TypeError`).
* *Implication:* Slicing is impossible.


2. **Distinct (Unique):** Duplicate values are automatically removed upon creation or addition.
3. **Mutable:** You can add or remove elements after creation.
4. **Heterogeneous:** You can mix data types (e.g., `int`, `str`, `tuple`) within a single set, **provided the elements are hashable**.

---

## 2. Creation Patterns

There are specific syntactic rules for creating sets, particularly when dealing with empty collections.

### A. Literal Syntax `{}`

The standard way to create a set with values.

```python
# Standard creation
permissions = {"read", "write", "execute"}

# Automatic Duplicate Removal
# The '8080' appears twice but is stored once
ports = {80, 443, 8080, 8080} 
print(ports) # Output: {80, 443, 8080}

```

### B. The `set()` Constructor

Used to convert other iterables into a set. This is the idiomatic way to remove duplicates from a list.

```python
# Convert List to Set (Deduplication)
raw_ids = [101, 102, 101, 103]
unique_ids = set(raw_ids) 
# Result: {101, 102, 103}

# Convert String to Set (Unique Characters)
chars = set("banana")
# Result: {'b', 'a', 'n'} (Order is random)

```

### C. The Empty Set Trap

**Critical:** Empty curly braces `{}` create an **empty Dictionary**, not a Set. To create an empty set, you *must* use `set()`.

```python
# ❌ Creates a Dictionary
empty_collection = {} 
print(type(empty_collection)) # <class 'dict'>

# ✅ Creates a Set
empty_set = set()
print(type(empty_set)) # <class 'set'>

```

---

## 3. The "Unordered" Reality

Beginners are often confused because when they print a set in Jupyter Notebooks or basic scripts, it *appears* sorted.

**This is an illusion.**

1. **Display Artifact:** Many shells/IDEs choose to display sets in sorted order for readability.
2. **Integer Hashing:** Small integers often hash to themselves, which can make them *appear* ordered in memory.

However, **never rely on this order**. It varies across Python versions, insertion histories, and execution environments.

```python
# The order you see is NOT guaranteed
s = {"Python", "Java", "C++"}

# Iteration order is arbitrary
for language in s:
    print(language) 
# Output might be: Java, Python, C++ (or any other permutation)

```

---

## 4. The Hashability Constraint

While a set itself is mutable, **every element inside a set must be immutable (hashable).**

Why? Because the set uses the element's **Hash Value** to determine its storage location. If an element (like a list) could change, its hash would change, and the set would lose track of it.

* **Allowed Elements:** Integers, Floats, Strings, Tuples, Booleans.
* **Forbidden Elements:** Lists, Dictionaries, other Sets.

```python
# ✅ Valid: Tuple is immutable
s = {1, (10, 20)} 

# ❌ Invalid: List is mutable
# s = {1, [10, 20]} 
# Raises TypeError: unhashable type: 'list'

```

---

## 5. Basic Operations

### Adding Elements

* **`add(element)`:** Adds a single element. If it exists, does nothing.

```python
s = {1, 2}
s.add(3)  # {1, 2, 3}

```

### Removing Elements

* **`remove(element)`:** Removes an item. Raises `KeyError` if not found.
* **`discard(element)`:** Removes an item. Does **nothing** if not found (safer).
* **`pop()`:** Removes and returns an **arbitrary** element. Since sets are unordered, you do not know which item will be popped.

```python
s = {"A", "B", "C"}

s.remove("A")  # Removes "A"
s.discard("Z") # Does nothing (No error)
item = s.pop() # Returns random element, e.g., "B"

```

---

## Summary Table

| Feature | List `[]` | Tuple `()` | Set `{}` |
| --- | --- | --- | --- |
| **Duplicates** | Allowed | Allowed | **Not Allowed** |
| **Order** | Ordered | Ordered | **Unordered** |
| **Indexing** | Yes (`L[0]`) | Yes (`T[0]`) | **No** |
| **Mutable** | Yes | No | **Yes** |
| **Elements** | Any Type | Any Type | **Immutable Only** |

# Python Sets: Internal Representation & Hashing

Unlike Lists or Tuples, which are represented as contiguous arrays of pointers accessible by a sequential index (), **Sets** use a **Hash Table** structure.

While a Set is backed by an array in memory, you cannot access elements via index because the position of an element depends on its **Hash Value**, not its insertion order. This architecture allows Sets to trade memory for speed, achieving **** performance for lookups and insertions.

---

## 1. The Hash Table Concept

A Hash Table is essentially a sparse array (an array with many empty slots). To determine where to store an element (Key), Python applies a **Hash Function**.

### The Formula

The fundamental logic for finding a slot index is:


### Example Trace (Size = 10)

Imagine a small hash table of size 10. We want to insert the integers: `5, 10, 23`.

1. **Insert 5:** `5 % 10 = 5`. Store `5` at Index 5.
2. **Insert 10:** `10 % 10 = 0`. Store `10` at Index 0.
3. **Insert 23:** `23 % 10 = 3`. Store `23` at Index 3.

**Resulting Memory Layout:**
| Index | 0 | 1 | 2 | 3 | 4 | 5 | ... | 9 |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **Value** | **10** | - | - | **23** | - | **5** | ... | - |

---

## 2. Handling Collisions

A **Collision** occurs when two distinct keys map to the same index.
*Example:* Insert `15` into the table above.
`15 % 10 = 5`.
Index 5 is already occupied by `5`.

### Collision Resolution Strategies

1. **Chaining (Transcript Model):** The slot at Index 5 becomes a pointer to a Linked List containing both `[5, 15]`.
2. **Open Addressing (CPython Reality):** Python actually uses **Open Addressing** (Linear Probing). If Index 5 is full, it calculates a new index (Probe) based on a perturbation formula and tries again until it finds an empty slot.

---

## 3. Performance Characteristics

Because the location of an element is determined by math rather than scanning, operations are incredibly fast.

| Operation | List (Array) | Set (Hash Table) | Why? |
| --- | --- | --- | --- |
| **Search** |  | **** | List must scan; Set jumps directly to index. |
| **Insert** |  /  | **** | List may need to shift items; Set writes to empty slot. |
| **Delete** |  | **** | List must shift items to close gap; Set just marks slot dummy. |

> **Note:** These are *Average Case* complexities. In the worst case (many collisions), a Set can degrade to , though Python's randomization makes this rare.

---

## 4. Why Sets are Unordered (The Resizing Effect)

In the lecture demonstration, adding the value `31` caused the entire print order of the set to change. This phenomenon is due to **Dynamic Resizing**.

### The Load Factor

Hash tables must remain sparse to maintain  speed. Python monitors the **Load Factor** (items / total slots).

* **Threshold:** When the table gets roughly  full.
* **Action:** Python allocates a new, larger table (e.g., increasing size from ).

### The Rehash

When the table size changes, the modulo divisor changes:

* **Old:** `x % 16`
* **New:** `x % 64`

This changes the index of *every* element. The elements are scattered into new positions, completely altering the iteration order.

### Demonstration

```python
# Small Set (Size 16 implied)
s = {10, 20, 50} 
# Order might be: 50, 20, 10 (Based on % 16)

# Trigger Resize
s.add(31) 
# Table expands to Size 64
# All keys rehashed with % 64
# New Order: 10, 50, 20, 31 (Completely different)

```

## Summary

* **Mechanism:** Sets use Hash Tables (Arrays + Hashing Function).
* **Order:** Determined by `hash(value) % size`. It appears random and changes when the set grows.
* **Speed:** Searching for an item is instant () because we calculate its address rather than searching for it.
* **Constraint:** Only **Hashable** (Immutable) objects can be stored, because if an object changed its value, its hash would change, and it would become "lost" in the wrong slot.

# Sets in Mathematics: Concepts and Operations

A **Set** is a fundamental concept in mathematics defined as a well-defined collection of distinct objects. These objects are called **elements** or **members** of the set. Understanding these mathematical principles is crucial because the `set` data type in Python is a direct implementation of this mathematical concept.

### 1. Core Properties

* **Distinctness:** A set cannot contain duplicate elements. The collection  is mathematically identical to .
* **Unordered:** The order of elements does not matter.  is the same set as .
* **Heterogeneity:** While sets often contain elements of the same type (e.g., integers), mathematically, a set can contain any type of distinct objects.

---

### 2. Set Relationships

We describe the relationship between two sets based on the elements they share.

* **Subset ():** Set  is a subset of Set  if *every* element of  is also present in .
* *Example:* If  and , then .
* *Note:* Every set is a subset of itself ().


* **Superset ():** If  is a subset of , then  is the superset of . It contains all elements of  plus potentially more.
* **Proper Subset ():** Set  is a proper subset of  if  is a subset of  AND  is not equal to  (meaning  has at least one extra element not in ).
* **Disjoint Sets:** Two sets are disjoint if they share **no common elements**.
* *Example:* If  and , they are disjoint because their intersection is empty.



---

### 3. Set Operations (Venn Diagrams)

Set operations allow us to combine or compare two sets to produce a new set. Venn diagrams are the standard visual tool for understanding these interactions.

#### A. Union ()

Combines all elements from both sets, removing duplicates.

* **Logic:** Elements present in Set  **OR** Set  (or both).
* **Example:** 
* **Commutative:** 

#### B. Intersection ()

Extracts only the elements common to both sets.

* **Logic:** Elements present in Set  **AND** Set .
* **Example:** 
* **Commutative:** 

#### C. Difference ()

Finds elements present in the first set but **not** in the second.

* **Logic (A - B):** Elements exclusively in  (remove any element that is also in ).
* **Example:** 
* **Non-Commutative:** 

#### D. Symmetric Difference ()

Finds elements present in either set, but **not** in both (the opposite of intersection).

* **Logic:** Elements exclusively in  combined with elements exclusively in .
* **Example:** 
* **Commutative:**

# Python Sets: Mathematical Operations

Python's `set` data type is a direct implementation of mathematical sets. This means we can perform standard set theory operations—like unions, intersections, and differences—natively in Python.

These operations are optimized in C, making them significantly faster than writing equivalent `for` loops to compare lists.

### The Two Modes of Operation

Most set operations in Python come in two flavors:

1. **Standard Method (e.g., `union()`):** Returns a **new set**. The original sets remain unchanged.
2. **Update Method (e.g., `update()`):** Modifies the set **in-place**. It returns `None`.

---

## 1. Union (OR Logic)

The **Union** of two sets combines all elements from both. Because sets are distinct, duplicates are automatically removed.

### Syntax

* **New Set:** `A.union(B)` or `A | B`
* **In-Place:** `A.update(B)` or `A |= B`

### Engineering Example

Merging permissions from two different user roles.

```python
# Setup
admin_perms = {"read", "write", "delete"}
guest_perms = {"read", "view_history"}

# 1. Standard Union (Create new set)
all_perms = admin_perms.union(guest_perms)

print(f"Combined: {all_perms}")
# Output: {'read', 'write', 'delete', 'view_history'}
# Note: "read" appears only once.

# 2. Update (Modify admin_perms directly)
admin_perms.update(guest_perms)
# admin_perms is now {'read', 'write', 'delete', 'view_history'}

```

---

## 2. Intersection (AND Logic)

The **Intersection** retrieves only the elements that exist in **both** sets simultaneously.

### Syntax

* **New Set:** `A.intersection(B)` or `A & B`
* **In-Place:** `A.intersection_update(B)` or `A &= B`

### Engineering Example

Finding common dependencies between two projects.

```python
proj_a_deps = {"pandas", "numpy", "requests"}
proj_b_deps = {"numpy", "flask", "requests"}

# Find common libraries
common = proj_a_deps.intersection(proj_b_deps)

print(f"Shared: {common}")
# Output: {'numpy', 'requests'}

```

---

## 3. Difference (Subtraction)

The **Difference** returns elements that are in the first set but **not** in the second.

* `A - B`  `B - A` (Order matters!)

### Syntax

* **New Set:** `A.difference(B)` or `A - B`
* **In-Place:** `A.difference_update(B)` or `A -= B`

### Engineering Example

Identifying missing tasks or revoked access.

```python
assigned_tasks = {"Task1", "Task2", "Task3", "Task4"}
completed_tasks = {"Task2", "Task4"}

# Find pending tasks (In Assigned, NOT in Completed)
pending = assigned_tasks.difference(completed_tasks)

print(f"Pending: {pending}")
# Output: {'Task1', 'Task3'}

```

---

## 4. Symmetric Difference (XOR Logic)

The **Symmetric Difference** returns elements that are in **either** set, but **not in both**. It is effectively the opposite of an intersection.

### Syntax

* **New Set:** `A.symmetric_difference(B)` or `A ^ B`
* **In-Place:** `A.symmetric_difference_update(B)` or `A ^= B`

### Engineering Example

Finding discrepancies between two data sources.

```python
source_a = {101, 102, 103}
source_b = {102, 103, 104}

# Find IDs that are unique to either source (discrepancies)
# 102 and 103 are ignored because they match.
diffs = source_a.symmetric_difference(source_b)

print(f"Discrepancies: {diffs}")
# Output: {101, 104}

```

---

## Summary Table

| Operation | Operator | Method (New Set) | Method (In-Place) | Description |
| --- | --- | --- | --- | --- |
| **Union** | `|` | `union()` | `update()` | All unique elements from both. |
| **Intersection** | `&` | `intersection()` | `intersection_update()` | Elements present in both. |
| **Difference** | `-` | `difference()` | `difference_update()` | Elements in A but not B. |
| **Sym. Diff** | `^` | `symmetric_difference()` | `symmetric_difference_update()` | Elements in A or B, not both. |

# Python Sets: Operators vs. Methods

In the previous section, we explored set operations using **Methods** (e.g., `.union()`, `.intersection()`). Python also supports **Operator Overloading** for these mathematical operations.

This allows you to write cleaner, more mathematical syntax using symbols (`|`, `&`, `-`, `^`). These operators function identically to their method counterparts but offer a more concise way to express set algebra.

> **Engineering Constraint:** unlike methods (which accept any iterable), set **operators** require **both** operands to be Sets. If you try `set_a | list_b`, Python will raise a `TypeError`.

---

## 1. Union (`|`)

The pipe operator `|` combines elements from two sets. It is equivalent to `A.union(B)`.

```python
s1 = {1, 2, 3, 5, 7}
s2 = {5, 7, 9, 10, 11}

# Operator Syntax
s3 = s1 | s2

print(s3) 
# Output: {1, 2, 3, 5, 7, 9, 10, 11}

```

---

## 2. Intersection (`&`)

The ampersand operator `&` retrieves elements common to both sets. It is equivalent to `A.intersection(B)`.

```python
s1 = {1, 2, 3, 5, 7}
s2 = {5, 7, 9, 10, 11}

# Operator Syntax
s3 = s1 & s2

print(s3)
# Output: {5, 7} (Only the common elements)

```

---

## 3. Difference (`-`)

The minus operator `-` removes elements found in the second set from the first. It is equivalent to `A.difference(B)`.

```python
s1 = {1, 2, 3, 5, 7}
s2 = {5, 7, 9, 10, 11}

# Operator Syntax (s1 minus s2)
s3 = s1 - s2

print(s3)
# Output: {1, 2, 3} (5 and 7 were removed because they exist in s2)

```

---

## 4. Symmetric Difference (`^`)

The caret operator `^` returns elements that are in **either** set, but **not both** (XOR operation). It is equivalent to `A.symmetric_difference(B)`.

```python
s1 = {1, 2, 3, 5, 7}
s2 = {5, 7, 9, 10, 11}

# Operator Syntax
s3 = s1 ^ s2

print(s3)
# Output: {1, 2, 3, 9, 10, 11} (Common items 5 and 7 are excluded)

```

---

## 5. In-Place Update Operators (Augmented Assignment)

Just as you can do `x += 1` for integers, you can use augmented assignment operators to modify sets **in-place**. This corresponds to the `_update` methods (e.g., `intersection_update`).

| Operation | Operator | Equivalent Method | Behavior |
| --- | --- | --- | --- |
| **Union Update** | `|=` | `.update()` | Adds elements from B to A. |
| **Intersection Update** | `&=` | `.intersection_update()` | Keeps only elements found in both. |
| **Difference Update** | `-=` | `.difference_update()` | Removes elements of B from A. |
| **Sym. Diff. Update** | `^=` | `.symmetric_difference_update()` | Updates A to have XOR elements. |

### Engineering Example: In-Place Modification

```python
# Initial Sets
active_users = {"Alice", "Bob"}
new_logins = {"Bob", "Charlie"}

# 1. Union Update (|=)
# "Add all new logins to active users"
active_users |= new_logins
# active_users is now {'Alice', 'Bob', 'Charlie'}

# Reset
active_users = {"Alice", "Bob"}

# 2. Difference Update (-=)
# "Remove anyone in new_logins from active_users"
active_users -= new_logins
# active_users is now {'Alice'} (Bob was removed)

```

---

## Summary Cheat Sheet

| Mathematical Logic | Method Name | Operator Symbol |
| --- | --- | --- |
| **OR** (Combine) | `union` | ` |
| **AND** (Common) | `intersection` | `&` |
| **SUBTRACT** (Remove) | `difference` | `-` |
| **XOR** (Unique to each) | `symmetric_difference` | `^` |

# Python Sets: Adding & Removing Elements

Because Sets are **unordered**, you cannot use indices (`s[0]`) or slicing (`s[0:2]`) to modify them. Instead, Python provides specific methods to add and remove elements based on their **value** or through arbitrary operations.

---

## 1. Adding Elements

These methods modify the set **in-place**.

### A. The `add()` Method

Adds a **single** element to the set.

* **Effect:** If the element already exists, it does nothing (no error).
* **Constraint:** You can add a tuple (immutable), but you cannot add a list (mutable).

```python
servers = {"192.168.1.1", "192.168.1.2"}

# Add a single item
servers.add("192.168.1.3")

# Adding a Tuple (treated as one item)
servers.add(("10.0.0.1", 8080))

print(servers)
# Output: {'192.168.1.3', '192.168.1.2', '192.168.1.1', ('10.0.0.1', 8080)}

```

### B. The `update()` Method

Adds **multiple** elements from an iterable (list, tuple, string, or another set).

* **Effect:** It "unpacks" the iterable and adds each item individually.

```python
inventory = {"apple", "banana"}
new_stock = ["cherry", "date", "apple"] # Note: "apple" is a duplicate

# Update unpacks the list
inventory.update(new_stock)

print(inventory)
# Output: {'banana', 'apple', 'cherry', 'date'}

```

---

## 2. Removing Elements

There are slight nuances in how these methods handle missing values.

### A. The `pop()` Method

Removes and returns an **arbitrary** element from the set.

* **Why arbitrary?** Because sets are unordered, you do not know which item is "first" or "last" in memory.
* **Error:** Raises `KeyError` if the set is empty.

```python
tickets = {101, 102, 103, 104}

# Remove an arbitrary item
processed = tickets.pop()
print(f"Processing ticket: {processed}")
# Could be 101, 104, etc. - It depends on the internal hash structure.

```

### B. `remove()` vs. `discard()`

Both methods accept a specific value to delete, but they handle "missing values" differently.

* **`remove(x)`:** Raises a `KeyError` if `x` is not found. Use this when the missing element indicates a bug in your logic.
* **`discard(x)`:** Does nothing if `x` is not found. Use this when you just want to ensure the element is gone, regardless of whether it was there to begin with.

```python
tags = {"python", "coding", "tutorial"}

# 1. Remove (Strict)
tags.remove("python") # Works
# tags.remove("java") # Raises KeyError: 'java'

# 2. Discard (Safe)
tags.discard("coding") # Works
tags.discard("java")   # Do nothing (No Error)

```

### C. The `clear()` Method

Removes **all** elements, leaving an empty set.

```python
cache = {0x1, 0x2, 0x3}
cache.clear()

print(cache) 
# Output: set() 
# Note: It prints 'set()', not '{}', to distinguish it from an empty dictionary.

```

---

## Summary Table

| Method | Argument | Action | Error Handling |
| --- | --- | --- | --- |
| **`add()`** | Single Item | Adds item. | No error on duplicate. |
| **`update()`** | Iterable | Adds multiple items. | No error on duplicates. |
| **`pop()`** | None | Removes random item. | `KeyError` if empty. |
| **`remove()`** | Value | Deletes value. | **`KeyError` if missing.** |
| **`discard()`** | Value | Deletes value. | **No error if missing.** |
| **`clear()`** | None | Deletes everything. | None. |